In [63]:
import csv, sqlite3

In [3]:
#CREATING THE CONNECTION & CURSOR
con = sqlite3.connect("chicago.db")
cur = con.cursor()

In [4]:
#VIEWING ALL THE TABLES IN THE DB
cur.execute('SELECT name from sqlite_master where type= "table"')
cur.fetchall() 

[('CHICAGO_SCHOOLS',), ('CHICAGO_CRIMES',), ('CHICAGO_CENSUS',)]

In [5]:
#Loading SQL Magic and the database
%load_ext sql
%sql sqlite:///chicago.db

In [6]:
%sql SELECT name, type FROM PRAGMA_TABLE_INFO('CHICAGO_CRIMES')

 * sqlite:///chicago.db
Done.


name,type
ID,INTEGER
CASE_NUMBER,TEXT
DATE,TEXT
BLOCK,TEXT
IUCR,INTEGER
PRIMARY_TYPE,TEXT
DESCRIPTION,TEXT
LOCATION_DESCRIPTION,TEXT
ARREST,TEXT
DOMESTIC,TEXT


In [7]:
#Number of crime records registered
%sql SELECT Count(*) FROM CHICAGO_CRIMES

 * sqlite:///chicago.db
Done.


Count(*)
533


In [9]:
#Community areas with per capita income less than 11000.
%sql SELECT COMMUNITY_AREA_NAME FROM CHICAGO_CENSUS WHERE PER_CAPITA_INCOME<11000 

 * sqlite:///chicago.db
Done.


COMMUNITY_AREA_NAME
West Garfield Park
South Lawndale
Fuller Park
Riverdale


In [10]:
#Listing case number of crimes involving minors
%sql SELECT CASE_NUMBER, DESCRIPTION FROM CHICAGO_CRIMES WHERE DESCRIPTION LIKE '%MINOR%' 

 * sqlite:///chicago.db
Done.


CASE_NUMBER,DESCRIPTION
HL266884,SELL/GIVE/DEL LIQUOR TO MINOR
HK238408,ILLEGAL CONSUMPTION BY MINOR


In [13]:
#Listing kidnapping crimes involving childs
%sql SELECT * FROM CHICAGO_CRIMES WHERE PRIMARY_TYPE='KIDNAPPING' and DESCRIPTION LIKE 'CHILD%'

 * sqlite:///chicago.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,FALSE,FALSE,1533,15,29,25,20,1143050,1897546,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


In [17]:
#Listing the type of crimes recorded at schools
%sql SELECT DISTINCT(PRIMARY_TYPE), DESCRIPTION FROM CHICAGO_CRIMES WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'

 * sqlite:///chicago.db
Done.


PRIMARY_TYPE,DESCRIPTION
BATTERY,SIMPLE
BATTERY,PRO EMP HANDS NO/MIN INJURY
CRIMINAL DAMAGE,TO VEHICLE
NARCOTICS,POSS: HEROIN(WHITE)
NARCOTICS,MANU/DEL:CANNABIS 10GM OR LESS
ASSAULT,PRO EMP HANDS NO/MIN INJURY
CRIMINAL TRESPASS,TO LAND
PUBLIC PEACE VIOLATION,BOMB THREAT


Listing the average safety score per each type of school.

In [43]:
%%sql 
SELECT avg(SAFETY_SCORE) as average_safety , ElementaryMiddleorHighSchool as school_type
FROM CHICAGO_SCHOOLS GROUP BY ElementaryMiddleorHighSchool

 * sqlite:///chicago.db
Done.


average_safety,school_type
49.52038369304557,ES
49.62352941176471,HS
48.0,MS


In [46]:
#Top 5 community with highest percentage of household poverty
%sql select * from chicago_census order by PERCENT_HOUSEHOLDS_BELOW_POVERTY desc limit 5

 * sqlite:///chicago.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_OF_HOUSING_CROWDED,PERCENT_HOUSEHOLDS_BELOW_POVERTY,PERCENT_AGED_16__UNEMPLOYED,PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA,PERCENT_AGED_UNDER_18_OR_OVER_64,PER_CAPITA_INCOME,HARDSHIP_INDEX
54,Riverdale,5.8,56.5,34.6,27.5,51.5,8201,98
37,Fuller Park,3.2,51.2,33.9,26.6,44.9,10432,97
68,Englewood,3.8,46.6,28.0,28.5,42.5,11888,94
29,North Lawndale,7.4,43.1,21.2,27.6,42.7,12034,87
27,East Garfield Park,8.2,42.4,19.6,21.3,43.2,12961,83


Community Area with the highest crime rate

In [54]:
%%sql 
select community_area_number, count(community_area_number) 
from chicago_crimes 
group by community_area_number 
order by count(community_area_number) desc LIMIT 1

 * sqlite:///chicago.db
Done.


COMMUNITY_AREA_NUMBER,count(community_area_number)
25,43


In [57]:
#Community area name with the highest hardship index
%sql select community_area_name from chicago_census where hardship_index=(select max(hardship_index) from chicago_census)

 * sqlite:///chicago.db
Done.


COMMUNITY_AREA_NAME
Riverdale


Community Area Name With the highest crime rate

In [59]:
%%sql 
select community_area_name from chicago_census where community_area_number=
(select community_area_number from chicago_crimes group by community_area_number order by 
count(community_area_number) desc LIMIT 1)

 * sqlite:///chicago.db
Done.


COMMUNITY_AREA_NAME
Austin
